### Header

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Audio
import pandas as pd
import numpy as np
import string
import re
from decimal import *
from datetime import datetime
import pickle
import os
import cProfile
import ast
import json

In [2]:
data_dict_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/Test_Data_Dict/'
#saved_data_html_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/HTML/'
os.chdir(data_dict_path)

csv_data_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/CSV_Data/'
file_name = 'Aggregated_Home_Data.csv'
#pd.DataFrame.to_csv(data, path_or_buf= csv_data_path + file_name)
#data = get_aggregatedData(os.listdir(data_dict_path)[:-1])
data = pd.read_csv(csv_data_path+file_name)
data = data.drop(axis=1, columns=['Year_Built:', 'Living_Area'])
data = data.set_index('House_Id')

## Main

###### Notes
1. Appliances: get the number of appliances included in sale, not the listing
2. Get the binary value of if price cut or not
3. Get Percentage away from listing from zestimate
4. Have Dataframe and do a merge from the main dataframe with the created data frames from the dict
    * Make a separate dictionary for each
    * If not listed initially, go forward and look for these:
        1. Zestimate
        2. Parking
        3. Heating / Cooling
        4. Home Type
5. Loop through Zip Code PKL files and find the top 50% - 75% of Dictionary Keys listed
    * Gather the top 50% - 75% and find the top values across the board again
6. Gather the Descriptions for each home and match them for Sentiment Analysis
    * Base the sentiment analysis on the price of the home or price per square foot, or price/Lot Size

###### Data Cleaning Notes
1. Try to mark empty lot based on:
    * absent bed/bath and year build
    * small number of photos: below the average
    * Home_Type
    * Large Living Area or Lot Size
2. Categorize These Columns:
    * Exterior Features
    * Parking Features
    * Home_Type
    * Foundation
    * Construction_Materials
    * Architectural_Style

In [3]:
data.Architectural_Style.value_counts()

Bungalow                                                           830
Colonial                                                           616
Ranch                                                              403
Upper/Lower Apartment                                              123
Other                                                               57
CAPE COD                                                            55
Tudor                                                               39
Common Entry Building                                               36
Historic                                                            32
Loft                                                                25
Contemporary                                                        19
Colonial, Historic                                                  17
1/2 Duplex with Land                                                15
Colonial, Tudor                                                     11
Quadru

In [31]:
data_cleaning_dict = {
    'Parking_Features': {
        'Garage': 'Garage',
        'Carport': 'Garage',
        'On Street': 'On street',
        'Off Street': 'Off street',
        'Assigned': 'Assigned',
        'Unassigned': 'Unassigned',
    },
    'Construction_Materials':{
        'Brick': ['Brick', 'Stone'],
        'Metal': ['Aluminum', 'Metal'],
        'Vinyl': ['Vinyl'],
        'Wood': ['Wood']
    }
}

In [17]:
data.Parking_Features.value_counts()

Attached Garage                                            1447
Detached Garage                                            1135
Attached Garage, Detached Garage                            429
None                                                         66
Off street                                                   65
On street                                                    26
Off street, On street                                        24
On Street                                                    19
1 Assigned Space                                             18
Carport                                                      11
Attached Garage, Off street                                   9
Detached Garage, On street                                    7
Attached Garage, On street                                    6
Carport, None                                                 5
Carport, Detached Garage                                      5
Detached Garage, Off street, On street  

In [38]:
dict_col = data_cleaning_dict['Parking_Features']
for i in range(0, len(temp.index)):
    temp_list = []
    list_type = False
    value_string = temp[i]
    if pd.isna(value_string):
        continue
    if ',' in temp[i]:
        value_string = value_string.split(',')
        print(value_string)
        list_type = True
    for key in dict_col.keys():
        if key in value_string:
            temp_list.append(key)
    print(temp_list)

['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Off street', ' On street']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
[]
['Attached Garage', ' On street']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
[]
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Attached Garage', ' Detached Garage']
[]
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['On Street']
['Attached Garage', ' Detached Garage']
[]
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
[]
['Ga

[]
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Carport', ' None']
[]
['Carport', ' None']
[]
[]
[]
['Carport', ' None']
[]
[]
['Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Unassigned', ' On Street']
['Unassigned']
['Assigned']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
[]
['Garage']
['Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['

['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
[]
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Assigned']
['Garage']
['Garage']
['Garage']
['Ga

['Garage']
['Garage']
['Garage']
['Off street', ' On street']
[]
['Garage']
['Attached Garage', ' Off street']
[]
['Garage']
['Garage']
[]
['Garage']
['Garage']
[]
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Attached Garage', ' Detached Garage']
[]
['Garage']
['Garage']
['Garage']
['Off street', ' On street']
[]
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Garage']
['Gar

In [34]:
def all_exist(mapping_values, string_value):
    return all(any(x in y for y in string_value) for x in mapping_values)
test = 'Attached Garage, Off street, On street'
test = test.split(',')
garage_feat = data_cleaning_dict['Parking_Features']['Garage']
all_exist(garage_feat, test)

False

In [28]:
test = 'Attached Garage, Off street, On street'
test = test.split(',')
print(test)
for value in test:
    #print('Garage' in value)
    print('On street' in value)

['Attached Garage', ' Off street', ' On street']
False
False
True


In [92]:
data.head().T

House_Id                                            2096667469_zpid  \
Bedrooms                                                          2   
Full_Bathrooms                                                    2   
Heating_Features                                         Forced air   
Total_Interior_Livable_Area                                    1182   
Number_Of_Appliances                                              8   
Bathrooms                                                         2   
Exterior_Features                                      Brick, Vinyl   
Parking_Features                                    Attached Garage   
Stories                                                           1   
Home_Type                                                 TOWNHOUSE   
Year_Built                                                     1905   
Foundation                                       Slab, Brick/Mortar   
Construction_Materials                          Vinyl Siding, Brick   
Architectural_Style                                        Historic   
New_Construction                                                 No   
Rent_Control                                                     No   
Structure_Type                                            Victorian   
Hoa_Fee                                                     $254/mo   
Average_School_Rating                                             0   
Average_School_Distance                                           0   
School_Listed_Binary                                          False   
Time_On_Zillow                                              55 days   
Living_Area                                                    2 bd   
Price                                                        549000   
Zestimate                                                    507316   
Price_Cut_Date                                                  NaN   
Number_Of_Photos                                                 21   
Address                      104 Edmund Pl APT 2, Detroit, MI 48201   
Views                                                           343   
Longitude                                                  -83.0535   
Latitude                                                    42.3442   
Saves                                                             5   
Price_Cut                                                       NaN   
Cooling                                                     Central   
Parking                                             Attached Garage   
Heating                                                  Forced air   
Type                                                      Townhouse   
Price_sqft                                                     $464   
Lot                                                         No Data   
Garage_Spaces                                                   NaN   
Has_Cooling                                                     NaN   
Parcel_Number                                                   NaN   
Lot_Size                                                        NaN   
Has_Hoa_Fee                                                     NaN   
Tax_Assessed_Value                                              NaN   
Annual_Tax_Amount                                               NaN   
Basement                                                        NaN   
Roof                                                            NaN   

House_Id                                            2090771520_zpid  \
Bedrooms                                                          2   
Full_Bathrooms                                                    2   
Heating_Features                                         Forced air   
Total_Interior_Livable_Area                                    1250   
Number_Of_Appliances                                              8   
Bathrooms                                                         2   
Exterior_Features      

In [123]:
temp = list(data.columns)
for i in range(0, len(temp)):
    temp[i] = temp[i].replace(':', '').replace('/', '_')
data.columns = temp
data.Bedrooms = force_numeric(data.Bedrooms, fill_value=0)
data.Bathrooms = force_numeric(data.Bathrooms, fill_value=0)
data.Stories = force_numeric(data.Stories, fill_value=0)
data.Year_Built = force_numeric(data.Year_Built, fill_value=None)
#data.Price_sqft = force_numeric(data.Price_sqft.str.replace('\$', '', regex=True), fill_value=0)
data.Price = force_numeric(data.Price, fill_value=None)
data['Calculated_Price_Sqft'] = (data.Price/data.Total_Interior_Livable_Area).replace([np.inf, -np.inf], np.nan)
data.Price_Cut = force_numeric(data.Price_Cut, fill_value=0)
#data.Time_On_Zillow = force_numeric(data.Time_On_Zillow.str.replace('\sdays', '', regex=True), fill_value=0)
#data.Lot_Size = force_numeric(data.Lot_Size.str.replace('\sacres', '', regex=True), fill_value=None)

#data.Total_Interior_Livable_Area = force_numeric(data.Total_Interior_Livable_Area.str.replace('\ssqft', '', regex=True).replace('\,', '', regex=True), fill_value=0)

In [112]:
data.Lot_Size = force_numeric(data.Lot_Size.str.replace('\sacres', '', regex=True), fill_value=None)

In [130]:
data[['Lot', 'Lot_Size']]
#data[data.Lot != data.Lot_Size]

Lot  Lot_Size
House_Id                              
2096667469_zpid      No Data       NaN
2090771520_zpid      No Data       NaN
2085043646_zpid          NaN       NaN
88380061_zpid            NaN      0.04
88335596_zpid            NaN       NaN
2083788163_zpid      No Data       NaN
88630896_zpid     1,830 sqft      0.04
2084082370_zpid      No Data       NaN
88743147_zpid     2,047 sqft      0.05
2083942797_zpid      No Data       NaN
2084845383_zpid      No Data       NaN
2137952874_zpid          NaN       NaN
88690280_zpid            NaN       NaN
88543710_zpid            NaN       NaN
2087322136_zpid      No Data       NaN
2091493449_zpid      No Data       NaN
2090578691_zpid      No Data       NaN
2091493220_zpid      No Data       NaN
2098645499_zpid          NaN       NaN
2089876009_zpid          NaN       NaN
88740052_zpid            NaN       NaN
88312802_zpid            NaN      0.04
2083617092_zpid      No Data       NaN
88186536_zpid            NaN      0.20
88282176_zpid            NaN       NaN
2106711152_zpid          NaN       NaN
2132478306_zpid          NaN       NaN
2084774752_zpid          NaN       NaN
2128541549_zpid          NaN       NaN
245780668_zpid    4,273 sqft      0.10
...                      ...       ...
88364008_zpid     5,663 sqft      0.13
88231214_zpid     6,534 sqft      0.15
88767868_zpid     0.89 acres      0.89
88458226_zpid     6,098 sqft      0.14
88505314_zpid     6,098 sqft      0.14
88591321_zpid     5,227 sqft      0.12
88456667_zpid      0.3 acres      0.30
88383835_zpid     6,534 sqft      0.15
88395199_zpid     0.56 acres      0.56
88113142_zpid     5,489 sqft      0.13
88546009_zpid     5,227 sqft      0.12
88098683_zpid    10,454 sqft      0.24
88747959_zpid    10,629 sqft      0.24
88324227_zpid     8,001 sqft      0.18
88517907_zpid     7,840 sqft      0.18
88696371_zpid     7,840 sqft      0.18
88427318_zpid     5,227 sqft      0.12
88085144_zpid     5,227 sqft      0.12
88756072_zpid     5,662 sqft      0.13
88115253_zpid     1,598 sqft      0.04
88423681_zpid     5,227 sqft      0.12
88599952_zpid     6,098 sqft      0.14
88266867_zpid     5,662 sqft      0.13
88242972_zpid     5,227 sqft      0.12
2084026497_zpid   7,405 sqft      0.17
88551193_zpid     6,098 sqft      0.14
88247117_zpid        No Data       NaN
2133759439_zpid    0.5 acres      0.50
88250628_zpid      0.5 acres      0.50
88209872_zpid     0.62 acres      0.62

[4421 rows x 2 columns]

In [ ]:
for val in list(data.Lot.values):
    if pd.isna(val):
        continue
    if val[-5:] == 'acres':
        # Remove Acres and convert to numeric
    if val[-4:] == 'sqft':
        # Remove sqft, convert to numeric and convert to acres

In [120]:
data[data[['Lot_Size', 'Total_Interior_Livable_Area']].Lot_Size > 10].T

House_Id                                              123210322_zpid  \
Bedrooms                                                           0   
Full_Bathrooms                                                   NaN   
Heating_Features                                          Forced air   
Total_Interior_Livable_Area                                     1148   
Number_Of_Appliances                                               0   
Bathrooms                                                          2   
Exterior_Features                                                NaN   
Parking_Features                                                 NaN   
Stories                                                            0   
Home_Type                                                      CONDO   
Year_Built                                                      1912   
Foundation                                                       NaN   
Construction_Materials                                           NaN   
Architectural_Style                                              NaN   
New_Construction                                                  No   
Rent_Control                                                      No   
Structure_Type                                                   NaN   
Hoa_Fee                                                            0   
Average_School_Rating                                              0   
Average_School_Distance                                            0   
School_Listed_Binary                                           False   
Time_On_Zillow                                             2569 days   
Living_Area                                                    -- bd   
Price                                                         384949   
Zestimate                                                     506512   
Price_Cut_Date                                                   NaN   
Number_Of_Photos                                                   3   
Address                      444 W Willis St # 10, Detroit, MI 48201   
Views                                                            151   
Longitude                                                   -83.0641   
Latitude                                                     42.3505   
Saves                                                              1   
Price_Cut                                                        NaN   
Cooling                                                      No Data   
Parking                                                      No Data   
Heating                                                   Forced air   
Type                                                           Condo   
Price_sqft                                                         0   
Lot                                                              NaN   
Garage_Spaces                                                    NaN   
Has_Cooling                                                       No   
Parcel_Number                                            02000889051   
Lot_Size                                                         148   
Has_Hoa_Fee                                                       No   
Tax_Assessed_Value                                          $255,400   
Annual_Tax_Amount                                                NaN   
Basement                                                         NaN   
Roof                                                             NaN   
Calculated_Price_Sqft                                        335.321   

House_Id                                             88119443_zpid  \
Bedrooms                                                         2   
Full_Bathrooms                                                   2   
Heating_Features                                        Forced air   
Total_Interior_Livable_Area                                   2397   
Number_Of_Appliances                             

In [111]:
for val in list(data.Lot_Size.values):
    if pd.isna(val):
        continue
    if val[-5:] != 'acres':
        print(val)

In [48]:
data.Total_Interior_Livable_Area.values[0][-4:]

'sqft'

## Functions

In [67]:
def load_Dictionary(path, file_name):
    with open(path + str(file_name), "rb") as input_file:
        data_dict = pickle.load(input_file)
    return data_dict
def make_series(name, series_name):
    return pd.Series(data=name, name=series_name)
def force_numeric(column, fill_value):
    if fill_value == None:
        return pd.to_numeric(column, errors='coerce')
    return pd.to_numeric(column, errors='coerce').fillna(fill_value)

def get_commonKeys(zip_code_file_list, percent_threshold):
    '''
    Return the keys that are at least covered in over half of the samples
    '''
    data_dict_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/Test_Data_Dict/'
    Overall_keys = []
    Interior_Details =[]
    Property_Details = []
    Construction_Details = []
    Building_Details = []
    Community_And_Neighborhood_Details = []
    Hoa_And_Financial_Details = []
    Other = []
    total_homes = 0
        
    for zip_code_file_name in zip_code_file_list:
        
        zip_code_dict = load_Dictionary(data_dict_path, zip_code_file_name)
        zip_code_keys = zip_code_dict.keys()
        num_homes = len(zip_code_keys)
        total_homes += num_homes
        for house in zip_code_keys:
            first_layer = zip_code_dict[house].keys()
            for val in first_layer: Overall_keys.append(val)
            if 'Interior_Details' in first_layer:
                for val in zip_code_dict[house]['Interior_Details'].keys(): Interior_Details.append(val)
            if 'Property_Details' in first_layer:
                for val in zip_code_dict[house]['Property_Details'].keys(): Property_Details.append(val)
            if 'Construction_Details' in first_layer:
                for val in zip_code_dict[house]['Construction_Details'].keys(): Construction_Details.append(val)
            if 'Building_Details' in first_layer:
                for val in zip_code_dict[house]['Building_Details'].keys(): Building_Details.append(val)
            if 'Community_And_Neighborhood_Details' in first_layer:
                for val in zip_code_dict[house]['Community_And_Neighborhood_Details'].keys(): Community_And_Neighborhood_Details.append(val)
            if 'Hoa_And_Financial_Details' in first_layer:
                for val in zip_code_dict[house]['Hoa_And_Financial_Details'].keys(): Hoa_And_Financial_Details.append(val)
            if 'Other' in first_layer:
                for val in zip_code_dict[house]['Other'].keys(): Other.append(val)
    series_list = [
        Overall_keys,
        Interior_Details,
        Property_Details,
        Construction_Details,
        Building_Details,
        Community_And_Neighborhood_Details,
        Hoa_And_Financial_Details,
        Other
    ]
    detail_keys = [
    'Overall_Keys',
    'Interior_Details',
    'Property_Details',
    'Construction_Details',
    'Building_Details',
    'Community_And_Neighborhood_Details',
    'Hoa_And_Financial_Details',
    'Other'
    ]
    #for i in range(0, len(series_list)):
    print(f'---------------- Total Homes Analyzed {total_homes} ----------------')
    for i in range(0, len(series_list)):
        temp = make_series(series_list[i], series_name=detail_keys[i])
        print('----------------'  + '----------------')
        temp = temp.value_counts().loc[temp.value_counts().values > total_homes* percent_threshold]
        display(temp.name, temp, temp.index)
        #series_list[i] = temp.value_counts().loc[temp.value_counts().values > num_homes* percent_threshold]
    #return series_list

def get_standardData(data_dict, house):
    standard_data_cols = [
    'Time_On_Zillow', 'Living_Area','Price', 'Zestimate','Price_Cut_Date',
    'Number_Of_Photos', 'Address', 'Views', 'Longitude','Latitude',
    'Saves', 'Price_Cut', 'Cooling:','Parking:', 'Heating:',
    'Year_Built:', 'Type:', 'Price/sqft:', 'Lot:'
    ]
    d = {k: data_dict[house].get(k, None) for k in standard_data_cols}
    
    return pd.DataFrame(d, index=[house])

def get_schoolData(data_dict, house):
    school_data = data_dict[house]['Schools']
    if school_data:
        school_data_listed = True
        average_rating = []
        average_distance = []
        for school in school_data.keys():
            rating = school_data[school]['Rating']
            distance = school_data[school]['Distance:']
            if rating == 'NA':
                rating = 0
            else:
                rating = np.array(int(re.match("\d", string=rating)[0]))
            distance = np.array(float(re.match("\d*\.?\d*", string=distance)[0]))
            average_rating.append(rating)
            average_distance.append(distance)
        average_rating = np.mean(np.array(average_rating))
        average_distance = np.mean(np.array(average_distance))
    else:
        school_data_listed = False
        average_rating = 0
        average_distance = 0
    d = {'Average_School_Rating': average_rating, 'Average_School_Distance': average_distance, 'School_Listed_Binary': school_data_listed}
    return pd.DataFrame(d, index=[house])
    
def get_aggregatedData(file_list):
    sub_DataColumns = {
    'Interior_Details': [
        'Total_Interior_Livable_Area', 'Heating_Features', 'Bathrooms',
        'Basement', 'Bedrooms', 'Full_Bathrooms', 'Has_Cooling', 'Number_Of_Appliances'
    ],
    'Property_Details': [
        'Exterior_Features', 'Lot_Size', 'Parcel_Number', 'Stories',
        'Parking_Features', 'Garage_Spaces'
    ],
    'Construction_Details': [
        'New_Construction', 'Home_Type', 'Year_Built', 'Roof', 'Foundation',
        'Construction_Materials', 'Architectural_Style','Major_Remodel_Year'
    ],
    'Building_Details': [
        'Rent_Control', 'Structure_Type'
    ],
    'Hoa_And_Financial_Details': [
        'Has_Hoa_Fee', 'Tax_Assessed_Value', 'Annual_Tax_Amount'
    ]
    }
    sub_detail_keys = [
        'Interior_Details',
        'Property_Details',
        'Construction_Details',
        'Building_Details',
        'Hoa_And_Financial_Details'
    ]
    cols = [
    'Time_On_Zillow', 'Living_Area', 'Average_School_Rating', 'Average_School_Distance',
    'Description_Score', 'Price', 'Zestimate', 'Price_Cut_Date','Hoa_Fee',
    'Number_Of_Photos', 'Address', 'Views', 'Longitude', 'Saves', 'Number_Of_Appliances',
    'Latitude', 'Price_Cut', 'Cooling:','Parking:', 'Heating:','Year_Built:', 'Type:', 'Price/sqft:', 'Lot:',
    'Other', 'Total_Interior_Livable_Area','Heating_Features', 'Bathrooms','Basement', 'Bedrooms', 'Full_Bathrooms', 'Has_Cooling',
    'Exterior_Features', 'Lot_Size', 'Parcel_Number', 'Stories','Parking_Features', 'Garage_Spaces',
    'New_Construction', 'Home_Type', 'Year_Built', 'Roof', 'Foundation', 'Construction_Materials',
    'Architectural_Style', 'Rent_Control', 'Structure_Type', 'Has_Hoa_Fee', 'Tax_Assessed_Value', 'Annual_Tax_Amount'
    ]
    data_dict_path = 'C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/Detroit/Saved_Data/Test_Data_Dict/'
    total_value = pd.DataFrame()
    total_value.index.name = 'House_Id'
    no_key_count = 0
    
    for file in file_list:
        data_dict = load_Dictionary(data_dict_path, file_name=file)
        print(f'Working on: {file}')
        for house in list(data_dict.keys()):
            row_value = pd.DataFrame(index=[house])
            row_value.index.name = 'House_Id'
            for sub_detail_key in sub_detail_keys:
                #print(sub_detail_key)
                #print(house)
                if sub_detail_key not in data_dict[house].keys():
                    no_key_count+=1
                    #print(f'---------------------------- {house} Has No Key: {sub_detail_key} ----------------------------')
                    temp_data = pd.DataFrame(columns=sub_DataColumns[sub_detail_key], index=[house])

                if sub_detail_key in data_dict[house].keys():
                    sub_data_values = data_dict[house][sub_detail_key]
                    temp_data = pd.DataFrame.from_dict(sub_data_values, orient='index').T
                    temp_data.index = [house]
                
                temp_data.index.name = 'House_Id'
                if sub_detail_key == 'Interior_Details':
                    if 'Appliances_Included_In_Sale' in temp_data.columns:
                        temp_data['Number_Of_Appliances'] = len(temp_data['Appliances_Included_In_Sale'].values[0].split(','))
                    else:
                        temp_data['Number_Of_Appliances'] = 0

                if sub_detail_key == 'Hoa_And_Financial_Details':
                    if 'Hoa_Fee' not in temp_data.columns:
                        temp_data['Hoa_Fee'] = 0
                #temp_data[sub_DataColumns[sub_detail_key]]
                intersection_cols = list(set.intersection(set(cols), set(temp_data.columns)))
                #display(temp_data[intersection_cols])
                #data = pd.merge(data, temp_data[intersection_cols], how='outer')
                #display(pd.merge(data, temp_data[intersection_cols], how='outer'))
                row_value = pd.concat([row_value, temp_data[intersection_cols]], axis=1, sort=False)
            #display(row_value.T)
            #print(f'House Complete: {house}')
            row_value = pd.concat([row_value, get_schoolData(data_dict, house)], axis=1, sort=False)
            row_value = pd.concat([row_value, get_standardData(data_dict, house)], axis=1, sort=False)
            total_value = pd.concat([total_value, row_value], axis=0, sort=False)
            #display(total_value)
    print(f'Number of missing keys: {no_key_count}')
    return total_value